In [1]:
using BenchmarkTools

In [2]:
function Convolution_2d(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d (generic function with 1 method)

In [3]:
function Conv_forward_v1(input, weights, bias)
    input_height, input_width, input_channels = size(input)
    kernel_height, kernel_width, _, output_channels = size(weights)

    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1
    output = zeros(Float32, output_height, output_width, output_channels)

    for k in 1:output_channels
        for c in 1:input_channels
            output[:, :, k] += Convolution_2d(input[:, :, c], weights[:, :, c, k]; bias=bias[k])
        end
    end
    
    return output
end

Conv_forward_v1 (generic function with 1 method)

In [4]:
function Conv_forward_v2(input, weights, bias)
    input_height, input_width, input_channels = size(input)
    kernel_height, kernel_width, _, output_channels = size(weights)

    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1
    output = zeros(Float32, output_height, output_width, output_channels)

    for k in 1:output_channels
        for c in 1:input_channels
            output[:, :, k] .+= Convolution_2d(input[:, :, c], weights[:, :, c, k]; bias=bias[k])
        end
    end
    
    return output
end

Conv_forward_v2 (generic function with 1 method)

In [5]:
function Conv_forward_v3(input, weights, bias)
    input_height, input_width, input_channels = size(input)
    kernel_height, kernel_width, _, output_channels = size(weights)

    output = zeros(Float32, input_height - kernel_height + 1, input_width - kernel_width + 1, output_channels)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    
    for k in 1:output_channels
        for c in 1:input_channels
            tmp_input .= @views input[:, :, c]
            tmp_weights .= @views weights[:, :, c, k]
            output[:, :, k] .+= Convolution_2d(tmp_input, tmp_weights; bias=bias[k])
        end
    end
    return output
end

Conv_forward_v3 (generic function with 1 method)

In [6]:
input = rand(Float32, 28, 28, 6);
weights = rand(Float32, 3, 3, 6, 16);
bias = rand(Float32, 16);

In [7]:
@benchmark Conv_forward_v1(input, weights, bias)

BenchmarkTools.Trial: 1621 samples with 1 evaluation.
 Range (min … max):  2.965 ms …   5.704 ms  ┊ GC (min … max): 0.00% … 44.76%
 Time  (median):     3.004 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.081 ms ± 180.066 μs  ┊ GC (mean ± σ):  2.10% ±  4.44%

  ▁▆█▇▆▄▂▂▃▃▃▁                       ▂▂▁▂▁                    ▁
  █████████████▇▆▆▆▄▅▄▄▁▄▄▅▄▁▄▁▁▇█▆█▇████████▇▄▇▇▆▅▇▆▆▆▄▅▄▅▆▅ █
  2.97 ms      Histogram: log(frequency) by time      3.58 ms <

 Memory estimate: 7.10 MiB, allocs estimate: 65378.

In [8]:
@benchmark Conv_forward_v2(input, weights, bias)

BenchmarkTools.Trial: 1667 samples with 1 evaluation.
 Range (min … max):  2.899 ms …   4.152 ms  ┊ GC (min … max): 0.00% … 22.77%
 Time  (median):     2.935 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.000 ms ± 136.559 μs  ┊ GC (mean ± σ):  1.71% ±  3.70%

  ▂▅█▇▅▄▃▂▃▃▁▂▂                      ▁▂▂▁▁▁▁                   
  ████████████████▇▇▆▅▅▄▄▅▁▅▁▄▄▁▁▄▇▇▆███████▇▇▇▆▆▆▄▄▆▅▆▅▁▄▅▁▅ █
  2.9 ms       Histogram: log(frequency) by time      3.43 ms <

 Memory estimate: 6.84 MiB, allocs estimate: 65282.

In [9]:
@benchmark Conv_forward_v3(input, weights, bias)

BenchmarkTools.Trial: 1694 samples with 1 evaluation.
 Range (min … max):  2.857 ms …   4.274 ms  ┊ GC (min … max): 0.00% … 19.99%
 Time  (median):     2.895 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.951 ms ± 129.427 μs  ┊ GC (mean ± σ):  1.61% ±  3.57%

  ▁▄▇█▇▆▄▄▃▂▁                          ▁ ▁▂▂▁▁                ▁
  ██████████████▆▇▇▇▇▆▅▅▆▅▄▄▁▅▄▄▁▁▁▅█▆██████████▇▇▇▇▇▇▅▅▅▄▄▄▄ █
  2.86 ms      Histogram: log(frequency) by time      3.35 ms <

 Memory estimate: 6.53 MiB, allocs estimate: 65092.